In [1]:
cd C:\Users\LIMDAESUN\EVT

C:\Users\LIMDAESUN\EVT


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.optimize import minimize
from scipy.optimize import root

In [3]:
data = pd.read_csv('F567C.s2020.HW5.data.csv', index_col=0)
data.columns = data.iloc[0]
data = data.iloc[1:]
data.index = pd.to_datetime(data.index)
data = data.astype(np.float64)
my_loss = -data.Close.pct_change().iloc[1:]

 ### MLE of GPD parameters

#### Generalized Pareto Distribution
$$
\begin{array}{c}
G(\xi, \beta, X, u) = 1 - ( 1+ \xi \frac{X-u}{\beta})^{-1/\xi} \\
g(\xi, \beta, X, u) = \frac{1}{\beta} ( 1 + \xi \frac{X-u}{\beta})^{- \frac{1}{\xi} -1}
\end{array}
$$
#### Maximum Likelihood Estimator
$$
\begin{array}{c}
 maximize \  \sum_{i=1}^{n} (ln \frac{1}{\beta} ( 1 + \xi \frac{X_i-u}{\beta})^{- \frac{1}{\xi} -1} )
\end{array}
$$

In [4]:
def log_likelihood(my_loss,beta_psi, u) :
    extreme_loss = my_loss[my_loss>u]
    likelihood = 1/beta_psi[0] * (1 + beta_psi[1]*(extreme_loss-u)/beta_psi[0])**(-1/beta_psi[1]-1) 
    return np.log(likelihood).sum()
def estimate_pareto_params(my_loss, u) : 
    #######################
    ##### initial value ###
    #######################
    beta = 0.006
    psi = 0.15
    beta_psi = np.array([beta,psi])    
    
    extreme_loss = my_loss[my_loss>u]

    def minus_log_likelihood(beta_psi) :
        return -1 *log_likelihood(my_loss,beta_psi,u)
        
    bnds = ((0.00001,10), (0.00001,10))
    x0 = (beta_psi[0],beta_psi[1])
    res = minimize(minus_log_likelihood, x0, method = 'SLSQP', bounds = bnds)    
    return res

 ### Probability function and VaR, ES

$$
\begin{array}{c}
\alpha = P(X>VaR) = P(X>u+x) \\ \\ = P(X>u)(1+\xi \frac{x}{\beta})^{-1/\xi} = P(X>u)(1+\xi \frac{VaR-u}{\beta})^{-1/\xi}
\\
\\
VaR = u + \frac{\beta}{\xi} ( (\frac{\alpha}{P(X>u)})^{-\xi}-1) \\ \\
ES  = \frac{VaR + \beta - \xi u}{1-\xi} 
\end{array}
$$

In [25]:
def conditional_density(X, beta_psi, u) :
    return 1/beta_psi[0]*(1 + beta_psi[1]/beta_psi[0] * (X-u))**(-1-1/beta_psi[1])

def cummulated_pareto(X,beta_psi,u) :
    return 1 - (1+beta_psi[1] * (X-u)/beta_psi[0]) ** ( -1/beta_psi[1])

def Prob(u,X,my_loss) :
    P_Excess_U = my_loss[my_loss>u].count() / my_loss.count()
    res = estimate_pareto_params(my_loss, u)
    return P_Excess_U * (1 - cummulated_pareto(X, res.x, u))

def VaR_EVT(my_loss, threshold = 0.01, confid_level = 0.999) :
    u = threshold
    alpha = 1-confid_level
    P_u = (my_loss>u).sum() / len(my_loss)
    res = estimate_pareto_params(my_loss, u)
    beta , psi = res.x[0] , res.x[1]
    VaR = u + beta/psi*( (alpha/P_u)**(-psi)-1 )
    return VaR

def ES_EVT(my_loss, threshold = 0.01, confid_level = 0.999) :
    u = threshold
    alpha = 1-confid_level
    P_u = (my_loss>u).sum() / len(my_loss)
    res = estimate_pareto_params(my_loss, u)
    beta , psi = res.x[0] , res.x[1]
    VaR = u + beta/psi*( (alpha/P_u)**(-psi)-1 )
    return (VaR + beta - psi * u)/(1-psi)


In [28]:
VaR_EVT(my_loss), ES_EVT(my_loss)

(0.061744150937680695, 0.07957654256739138)